<a href="https://colab.research.google.com/github/Iaquine/Implementa-o-K-means-dataset-HAR/blob/master/Implementa%C3%A7%C3%A3o_do_K_means_com_o_dataset_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>